In [9]:
import pandas as pd
import numpy as np
import math

In [8]:
def I_t(input_file, electrode_area, names):
    df = pd.read_csv(input_file, names = names)
    starting_row = 0
    row_list = df.index.tolist()
    for row in row_list:
        try:
            float(df.loc[row, names[0]])
            break
        except ValueError:
            starting_row += 1
    df = pd.read_csv(input_file, header = starting_row-1, index_col = False, names = names)
    df['Current density ($mA/cm^2$)'] = df[names[1]] * 1000 / electrode_area
    return df

In [7]:
def LSV(input_file, electrode_area, names):
    df = pd.read_csv(input_file, names = names)
    starting_row = 0
    row_list = df.index.tolist()
    for row in row_list:
        try:
            float(df.loc[row, names[0]])
            break
        except ValueError:
            starting_row += 1
    df = pd.read_csv(input_file, header = starting_row-1, index_col = False, names = names)
    df['Potential (V vs RHE)'] = df[names[0]] + 0.0592 * 13.6 + 0.1976
    df['Current density ($mA/cm^2$)'] = df[names[1]] * 1000 / electrode_area
    return df

In [10]:
def IMPE(input_file):
    with open (input_file, 'r') as f:
        starting_row = 1
        lines = f.readlines()
        for line in lines:
            if 'Potential/V' in line:
                print(f'Found, starting row is {starting_row}')
                break
            elif line == '\n':
                print('Blank line')
            else:
                starting_row += 1
            
    df = pd.read_csv(input_file, header = starting_row-1, sep = '\t')
    df[['Potential/V', 'Z\'/ohm', 'Z\"/ohm', 'Z/ohm', 'Phase/deg']] = df['Potential/V, Z\'/ohm, Z\"/ohm, Z/ohm, Phase/deg'].str.split(pat = ', ', expand = True)
    df.drop('Potential/V, Z\'/ohm, Z\"/ohm, Z/ohm, Phase/deg', axis = 1, inplace = True)
    df = df.astype(float)
    df['$1/C^2$'] = 4 * (math.pi**2) * 10**8 * (df['Z\"/ohm']**2)
    df['Potential (V vs RHE)'] = df['Potential/V'] + 0.198 + 0.0592*13
    df = df[['Potential (V vs RHE)', '$1/C^2$']]
    index_list = df.index.tolist()
    index_list

    for index in index_list:
        if df.apply(lambda x: x.is_monotonic)[1] == True:
            break
        else:
            df.drop(labels = [index], axis = 0, inplace = True)

    df.reset_index(drop = True, inplace = True)
    return df

In [14]:
def fitting(df, init):
    coef = np.polyfit(df.loc[init:, 'Potential (V vs RHE)'], df.loc[init:, '$1/C^2$'], 1)
    poly_fn = np.poly1d(coef)
    return coef, poly_fn